# Pocket Alternative

[Pocket is shutting down](https://support.mozilla.org/en-US/kb/future-of-pocket)

I don't trust that anyone else who has alternatives is going to be able to necessarily be around forever so I guess I have to make it myself.

## Setup

### Project Setup
To set up the development environment:

This project uses [uv](https://docs.astral.sh/uv/getting-started/installation/) for managing all things Python including notebooks.

## Requirements
So here's what I want

### Goals
- A browser extension that I can trigger on any link
- An Android app that I can share links to
- The ability to tag links
- The ability to search stored links searching across both tags and text
- Keep costs low. We can use AWS or something, but I want to stay within the free tier as much as possible
- Some sort of monitoring

### Nice-to-haves
- Store the full article text
- Ability to pierce paywalls somewhat
- Use the archive link if the full link has a paywall
- AI-enabled search via embeddings
- CI/CD

### Non-goals
- Dealing with the app store, I'll side-load the app if need be
- Multi-tenancy, I'm ok with just a single account for the entire instance
- Scale

## Setup

### AWS Account Configuration

#### SSO
Follow the steps in the jupyter notebook here to set it up [here.](./aws-configuration.ipynb)